# Background

Em primeiro lugar, vamos observar o enunciado com atenção. Se seguida está um resumo das partes importantes dele.

## Contexto
> Pretende-se que seja implementada uma solução computacional para estudo e análise de dados de grande dimensão. Nesse sentido, deverá ser contruído um modelo de análise e processamento de dados baseado em métodos e algoritmos
lecionados nas aulas

> A escolha do domínio de dados e respetico conjunto de dados a utilizar, bem [como] a formulação do próprio problema em estudo, será da responsabilidade dos autores do trabalho, [... que ...] devem ser obtidos a partr da lista de fontes especificada [...] na tabela 1

Entre as opções, nós escolhemos a competição "Predict Student Performance from Game Play" pois parecia um tema com potencial, e parece alcançável com os métodos lecionados em aula, conforme o objetivo do projeto.

## Requesitos
> [...] requisito de utilização de Apache Spark e Python é mandatório
>
> [...] vão ser utilizados serviços em ambiente cloud, da Amazon para contexto académico – AWS Academy – na qual será disponibilizada uma classe de ensino para os(as) alunos(as) da unidade curricular
>
> Os algorimos a utilizar que estarão na base da construção do [... MAPD] devem fazer parte da plataforma Apache Spark
>
> A implementação da dolução deve ser [...] composta por mais do que um notebook ou módulo Python. Compete aos autores do trabalho a estruturar [...] o código implementado.
>
> A submissão consiste num arquivo em formato zip [...] com [...] (i) relatório e (ii) notebooks e/ou módulos Python.
>
> relatório deve ser um documento sucinto e em formato pdf, com o máximo de oito [8] páginas.
>
> O prazo de submissão é 12:00 de 8 de abril de 2023 [...].


Nesta secção, vamos observar os dados que temos de forma geral, e o objetivo da competição do kaggle.

Conforme o enuncuiado do kaggle:

> The goal of this competition is to predict student performance during game-based learning in real-time.

Nós temos acesso a uma base de dados de todos os eventos de cada sessão de um utilizador, e cada sessão tem 18 questões, onde nós sabemos se a sessão acertou em cada sessão. O nosso objetivo é prever, baseado nos movimentos e nas ações de uma sessão, se essa vai acertar ou errar nas questões. Vamos chamar a essas questões $q _\lambda, \lambda \in \left[ 1, 18 \right]$. O jogo parece apresentar 22 níveis, com 3 "checkpoints" de questões, um no nível 4, outro no nível 12 e o último no último nível, 22. Vamos chamar a esses níveis $n_\lambda, \lambda \in \left [1, 22\right]$. Para cada sessão $s$, temos que prever se $s$ vai acertar na questão, baseado nas ações prévias da sessão.

Para entender melhor a base de dados, nós planeamos fazer dois passos:

* Jogar o jogo.
* Interpretar os dados que nos dão.

## Jogar o jogar

O jogo encontra-se em https://pbswisconsineducation.org/jowilder/play-the-game/, e tem o título de **Jo Wilder and the Capitol Case**, produzido por *PBS Winsconsin Education*. 

## Interpretar os dados que nos dão

In [ ]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark
from pyspark.sql import SparkSession
import pyspark.pandas as ps

In [ ]:
spark = SparkSession.builder \
                    .master('local[1]') \
                    .appName('pbd') \
                    .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
! start http://localhost:4040/jobs/

Ao executar `kaggle c download -p .\data\predict-student-performance-from-game-play`, e exportar o zip resultado, são exportados 4 ficheiros csv: 
* test.csv - O ficheiro do conjunto de teste 
* train.csv - O ficheiro do conjunto de treino
* train_labels.csv - O ficheiro com as respostas das sessões
* sample_submission - O ficheiro com exemplos de resposta

Este último é de nos pouca importância neste passo, os outros vamos observar 

In [ ]:
paths = [r".\data\test.csv", r".\data\sample_submission.csv", r".\data\train_labels.csv", r".\data\train.csv"]
df_test, df_sample_submissions, df_train_labels, df_train = [
    ps.read_csv(i, index_col = None)
    for i in paths
]

C:\Users\Plancha\scoop\apps\python\current\Lib\site-packages\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [ ]:
df_test.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,session_level
0,20090109393214576,0,0,cutscene_click,basic,0,NaN,-413.991405,75.685314,380.0,259.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4,None
1,20090109393214576,1,1965,person_click,basic,0,NaN,-105.991405,-63.314686,688.0,398.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,None
2,20090109393214576,2,3614,person_click,basic,0,NaN,-418.991405,47.685314,375.0,287.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,None
3,20090109393214576,3,5330,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,392.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,None
4,20090109393214576,4,6397,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,392.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,None
